In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./PSI_METRICS/credit_score.csv")
df.head(5)

,CUST_ID,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT
0,C02COQEVYU,33269,0,532304,0.0000,16.0000,1.2000,1889,945,0.5003,...,0.0000,0.0625,High,1,0,0,0,0,444,1
1,C02OZKC0ZF,77158,91187,315648,1.1818,4.0909,3.4615,5818,111,0.0191,...,0.7692,0.2222,No,1,0,0,1,0,625,0
2,C03FHP2D0A,30917,21642,534864,0.7000,17.3000,24.7142,1157,860,0.7433,...,1.4286,0.0578,High,1,0,0,1,0,469,1
3,C03PVPPHOY,80657,64526,629125,0.8000,7.8000,9.7499,6857,3686,0.5376,...,1.2500,0.1282,High,1,0,0,1,0,559,0
4,C04J69MUX0,149971,1172498,2399531,7.8182,16.0000,2.0465,1978,322,0.1628,...,0.1163,0.0568,High,1,1,1,1,1,473,0


In [3]:
new_df = df[["INCOME","SAVINGS", "DEBT", "CREDIT_SCORE", "DEFAULT"]]
new_df.head(5)

,INCOME,SAVINGS,DEBT,CREDIT_SCORE,DEFAULT
0,33269,0,532304,444,1
1,77158,91187,315648,625,0
2,30917,21642,534864,469,1
3,80657,64526,629125,559,0
4,149971,1172498,2399531,473,0


In [4]:
X = new_df.drop(["DEFAULT"], axis=1)
y = new_df["DEFAULT"]
print(X.shape, y.shape)

(1000, 4) (1000,)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [45]:
def PSI(y_test, y_pred, n_bins = 10):
    hist, bin_edges = np.histogram(y_test, n_bins)
    hist2, b_edges = np.histogram(y_pred, bins = bin_edges)
    dev = np.array(hist)
    dev = dev / len(y_test)
    exp = np.array(hist2)
    exp[exp == 0] = 1
    exp = exp / len(y_pred)
    psi = (dev - exp) * np.log(dev / exp)

    return b_edges, dev, exp, psi

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10, random_state=45)

In [12]:
hist, bin_edges = np.histogram(new_df["CREDIT_SCORE"], 10)
print(hist, bin_edges)

[  5  14  16  49 143 293 357 112  10   1] [300. 350. 400. 450. 500. 550. 600. 650. 700. 750. 800.]


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [28]:
y_pred

array([[0.1, 0.9],
       [0.7, 0.3],
       [0.2, 0.8],
       ...,
       [0.3, 0.7],
       [0.8, 0.2],
       [0.8, 0.2]])

In [39]:
from scipy.stats import ks_2samp
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
ks_df = pd.DataFrame({
    "p": y_pred[:,1],
    "y": y_test
})
ks, p_value = ks_2samp(ks_df.loc[ks_df.y==0,"p"], ks_df.loc[ks_df.y==1,"p"])
print(ks, p_value)
ks_df = ks_df.sort_values(by="p", ascending=False).reset_index(drop=True)
ks_df.to_csv("KS.csv")

0.20426941479573058 0.05731712663702975


In [42]:
freq, bin_edges = np.histogram(ks_df["p"].to_numpy(), 10)
bin_edges

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [67]:
y_pred.shape

(200,)

In [46]:
bins, dev, exp, psi = PSI(y_test, y_pred)
print(bins)
print(dev)
print(exp)
print(psi)
print(sum(psi))

[300. 344. 388. 432. 476. 520. 564. 608. 652. 696. 740.]
[0.01  0.01  0.015 0.015 0.07  0.195 0.225 0.365 0.08  0.015]
[0.005 0.01  0.055 0.03  0.13  0.235 0.28  0.195 0.06  0.005]
[0.00346574 0.         0.05197132 0.01039721 0.03714235 0.00746344
 0.01202791 0.10657263 0.00575364 0.01098612]
0.24578034924596814


In [73]:

def calculate_psi(expected, actual, n_bins=10):
    """
    Calculates the Population Stability Index (PSI) for a single variable.

    Args:
        expected (pd.Series or np.array): The baseline distribution (e.g., training data).
        actual (pd.Series or np.array): The new distribution to compare against.
        n_bins (int): The number of bins to use for the calculation.

    Returns:
        float: The calculated PSI value.
    """
    # 1. Ensure inputs are Series for easier handling
    if not isinstance(expected, pd.Series):
        expected = pd.Series(expected)
    if not isinstance(actual, pd.Series):
        actual = pd.Series(actual)

    # 2. Bin the expected data based on quantiles to get breakpoints
    breakpoints = np.percentile(expected, np.arange(0, 101, 100/n_bins))
    breakpoints = np.unique(breakpoints)

    # 3. Assign data to bins and calculate percentages
    expected_binned = pd.cut(expected, bins=breakpoints, include_lowest=True, right=True, labels=False)
    actual_binned = pd.cut(actual, bins=breakpoints, include_lowest=True, right=True, labels=False)

    expected_counts = expected_binned.value_counts(normalize=True).sort_index()
    actual_counts = actual_binned.value_counts(normalize=True).sort_index()

    # Create a DataFrame for the PSI calculation
    psi_df = pd.DataFrame({'Expected': expected_counts, 'Actual': actual_counts}).fillna(0)

    # 4. Handle zeros by clipping values to a small epsilon
    psi_df['Expected'] = np.clip(psi_df['Expected'], 0.0001, None)
    psi_df['Actual'] = np.clip(psi_df['Actual'], 0.0001, None)

    # 5. Apply the PSI formula
    psi_df['PSI'] = (psi_df['Actual'] - psi_df['Expected']) * np.log(psi_df['Actual'] / psi_df['Expected'])

    return psi_df['PSI'].sum()


psi_value = calculate_psi(y_pred, y_test)
print(f"The calculated PSI value is: {psi_value:.4f}")

The calculated PSI value is: 0.7813


In [75]:
dic = {"Credit_Score": y_test,
       "Pred_Credit_Score": y_pred  
}
df1 = pd.DataFrame(dic)
df1.to_csv("PSI.csv")

In [ ]:
dic = {
    "score_range": bins,
    "scoring%": exp * 100,
    "training%": dev * 100,
}